In [2]:
import boto3
boto3.client('sts').get_caller_identity()

{'UserId': 'AROAV52AN3HJJIZVUDZ2F:SageMaker',
 'Account': '407620147666',
 'Arn': 'arn:aws:sts::407620147666:assumed-role/role-ap-segemaker-execution-setopdata2.0-t/SageMaker',
 'ResponseMetadata': {'RequestId': 'f9894023-72a9-4fb3-b6bf-e97736f61613',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f9894023-72a9-4fb3-b6bf-e97736f61613',
   'content-type': 'text/xml',
   'content-length': '467',
   'date': 'Wed, 09 Aug 2023 14:32:54 GMT'},
  'RetryAttempts': 0}}

In [5]:
# Create session using your current creds
boto_sts=boto3.client('sts')

# Request to assume the role like this, the ARN is the Role's ARN from 
# the other account you wish to assume. Not your current ARN.
stsresponse = boto_sts.assume_role(
    RoleArn="arn:aws:iam::354925255288:role/test",
    RoleSessionName='newsession'
)

# Save the details from assumed role into vars
newsession_id = stsresponse["Credentials"]["AccessKeyId"]
newsession_key = stsresponse["Credentials"]["SecretAccessKey"]
newsession_token = stsresponse["Credentials"]["SessionToken"]


In [ ]:
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [8]:
sm_client = boto3.client(
    'sagemaker',
    region_name='us-east-1',
    aws_access_key_id=newsession_id,
    aws_secret_access_key=newsession_key,
    aws_session_token=newsession_token
)

In [9]:
smr_client = boto3.client(
    'sagemaker-runtime',
    region_name='us-east-1',
    aws_access_key_id=newsession_id,
    aws_secret_access_key=newsession_key,
    aws_session_token=newsession_token
)

In [10]:
endpoint_name = 'huggingface-pytorch-tgi-inference-2023-08-09-14-15-39-283'

In [15]:
import sagemaker

In [29]:
sagemaker_session = sagemaker.session.Session(
    sagemaker_client=sm_client, 
    sagemaker_runtime_client=smr_client)

In [52]:
from sagemaker import serializers

In [63]:
from sagemaker import deserializers

In [64]:
llm = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name, 
    sagemaker_session=sagemaker_session, 
    serializer=serializers.JSONSerializer(),
    deserializers=deserializers.JSONDeserializer())

In [65]:
def build_llama2_prompt(messages):
    startPrompt = "<s>[INST] "
    endPrompt = " [/INST]"
    conversation = []
    for index, message in enumerate(messages):
        if message["role"] == "system" and index == 0:
            conversation.append(f"<<SYS>>\n{message['content']}\n<</SYS>>\n\n")
        elif message["role"] == "user":
            conversation.append(message["content"].strip())
        else:
            conversation.append(f" [/INST] {message.content}</s><s>[INST] ")

    return startPrompt + "".join(conversation) + endPrompt



In [66]:
instruction = '''
Assign appropriate labels/tags to the product as "tags", in flavor, brand, key ingredient, and package size 
(if applicable), etc.  

product: {product}

1. Return the results in JSON format with the following key: "tags".
2. Replied answer should be as diverse as possible.
3. Do not repeat answers.
4. Reply in Taiwan Chinese.
5. Please avoid choosing duplicate tags.
6. Your tags shall be no more than {max_tags}.
7. Reply tag json only, no more information.

The json result is: 
'''


In [67]:
messages = [
  { "role": "system","content": instruction.format(product='統一蜜豆奶', max_tags=4)}
]


In [68]:
messages

[{'role': 'system',
  'content': '\nAssign appropriate labels/tags to the product as "tags", in flavor, brand, key ingredient, and package size \n(if applicable), etc.  \n\nproduct: 統一蜜豆奶\n\n1. Return the results in JSON format with the following key: "tags".\n2. Replied answer should be as diverse as possible.\n3. Do not repeat answers.\n4. Reply in Taiwan Chinese.\n5. Please avoid choosing duplicate tags.\n6. Your tags shall be no more than 4.\n7. Reply tag json only, no more information.\n\nThe json result is: \n'}]

In [71]:
# define question and add to messages
# instruction = "What are some cool ideas to do in the summer?"
# messages.append({"role": "user", "content": instruction})
prompt = build_llama2_prompt(messages)

# chat = llm.predict({"inputs": json.dumps(prompt)})
chat = llm.predict({"inputs":prompt})
print(json.loads(chat)[0]["generated_text"][len(prompt):])


 {
"tags": [
"蜜豆奶",



In [73]:
# hyperparameters for llm
payload = {
  "inputs":  prompt,
  "parameters": {
    "do_sample": True,
    "top_p": 0.6,
    "temperature": 0.01,
    "top_k": 50,
    "max_new_tokens": 100,
    "repetition_penalty": 1.03,
    "stop": ["</s>"]
  }
}

# send request to endpoint
response = llm.predict(payload)
chat = json.loads(response)
print(chat[0]["generated_text"][len(prompt):])


 {
"tags": [
"蜜豆奶",
"統一",
"牛奶",
"甜點"
]
}
